# Assignment 2 - Titanic Dataset

## Part 1: Exploratory Analysis

We will first proceed to import the pandas package and then use the read_csv function to read the dataset. We have also set the PassengerId as the index column for this dataset as it seemed the most appropriate to use.

In [1]:
import pandas as pd
import numpy as np

In [2]:
titanic_df = pd.read_csv(
    "C:/Users/asus/Desktop/Python Programming course/IE Python/train.csv",
    index_col="PassengerId"
)
titanic_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We have calculated the survival percentage by first taking a sum (since Survival is 1 and dead is 0) of the Survival column and dividing that the number of rows in the survival column.

In [5]:
Survival_Percentage = (titanic_df['Survived'].sum()/titanic_df['Survived'].count())
print('The percentage of people who survived is', round(Survival_Percentage *100,2),'%')

The percentage of people who survived is 38.38 %


To get the number of males and females in the dataset, we can create a variable to store each the number of each sex so that it can then be used for later calculations.

In [6]:
male_female = titanic_df.groupby(["Sex"]).size()
male_female

Sex
female    314
male      577
dtype: int64

We then calculate the percent of male passengers in the dataset by dividing by the total passenger. We do the same to get the value for the percent of female passengers.

In [7]:
male_pass = male_female['male'] / (male_female['male'] + male_female['female'])
print('The pecentage of male passengers is', round(male_pass * 100, 2),'%')

The pecentage of male passengers is 64.76 %


In [8]:
female_pass = male_female['female']/ (male_female['male'] + male_female['female'])
print('The pecentage of female passengers is', round(female_pass * 100, 2))

The pecentage of female passengers is 35.24


To find the number of survivors by split by Sex, we use a group by function on Sex and Survived

In [9]:
m_f_survived = titanic_df.groupby(["Sex", "Survived"]).size()
m_f_survived

Sex     Survived
female  0            81
        1           233
male    0           468
        1           109
dtype: int64

We can then use the above dataframe to index and calculate the number of female and male survivors respectively uaing idexation methods.

In [10]:
f_survived = m_f_survived["female"][1]
female_survived_perc = f_survived / (f_survived + m_f_survived["female"][0])
print('The percentage of female survivors is', round(female_survived_perc * 100, 2),'%')

The percentage of female survivors is 74.2 %


In [11]:
m_survived = m_f_survived["male"][1]
male_survived_perc = m_survived / (m_survived + m_f_survived["male"][0])
print('The percentage of male survivors is', round(male_survived_perc * 100, 2),'%')

The percentage of male survivors is 18.89 %


To find the proportion of survivors that are male and female, we can first create a variable to find all the survivors i.e. have a 1 in the Survived column, and use this to calculate the respective survival proportions for both men and women in the data set. We use the len function to count the number of records that fulfill each condition.

In [12]:
# The Number of B

survived_true = titanic_df['Survived'] == 1
survivors_m = len(titanic_df[(titanic_df['Survived'] == 1) & (titanic_df['Sex'] == 'male')]) / len(survived_true[survived_true == True].index)
survivors_f = len(titanic_df[(titanic_df['Survived'] == 1) & (titanic_df['Sex'] == 'female')]) / len(survived_true[survived_true == True].index)

print('Of the Survivors', round(survivors_m * 100, 2),'% are male and', round(survivors_f * 100, 2),'% are female.')

Of the Survivors 31.87 % are male and 68.13 % are female.


To create the 2 by 2 dataframe of the male and female survivor rates, we first calculate each variable and then insert into an dataframe. We can specify the necessary columns and also the row index to ensure readability.

In [13]:
prob_m_surv = male_survived_perc * male_pass
prob_f_surv = female_survived_perc * female_pass
prob_m_nsurv = (1-male_survived_perc) * male_pass
prob_f_nsurv = (1-female_survived_perc) * female_pass

data = [[prob_m_surv, prob_m_nsurv], [prob_f_surv, prob_f_nsurv]]

survival_table = pd.DataFrame(data, columns = ['Survived', 'Not Survived'], index = ['Male', 'Female'])

(100 * survival_table).round(1)

,Survived,Not Survived
Male,12.2,52.5
Female,26.2,9.1


We then calculate the different probabilities of each class, split by survived and non-survived and by sex. We can do this by using a unstack function to group each sex by each of the separate classes that we wish to compare them by.

In [14]:
class_survival = round(titanic_df.groupby(["Pclass", "Sex", "Survived"]).size().unstack()/titanic_df.Survived.count(), 4)
cl_surv = class_survival.rename(columns={0: 'Not Survived', 1: 'Survived'})
(100 * cl_surv).round(1)

Survived       Not Survived  Survived
Pclass Sex                           
1      female           0.3      10.2
       male             8.6       5.1
2      female           0.7       7.9
       male            10.2       1.9
3      female           8.1       8.1
       male            33.7       5.3

## Part 2: Imputing the missing values of the Age column

For the age column, to impute values we need to use appropriate logic to estimate the values that can be used to replace the missing values in the age column. To first get a sense of the missing data, we need to know how many values are missing from the age column. We can do this by using the isnull and sum methods. There are 177 null values which is quite significant. Hence we need to impute some values for this column.

In [15]:
titanic_df.Age.isnull().sum()


177

I choose the median as the best measure as it is least affected by outlier values as that's highly possible in this dataset as we had both old people as well as children. A good starting point would be to use the median age of passengers split by Passenger class ('Pclass') and Sex ('Sex'). It's first a good idea to check if there are any null values in these columns as if there are, it wouldn't prove a useful analysis.

In [16]:
titanic_df.Sex.isnull().sum()

0

In [17]:
titanic_df.Pclass.isnull().sum()

0

We can proceed to calculate the median age by using a combination of indexation and a groupby method to split this calculation on the dataframe. Once we obtain our results, we can see that the median age for each class split by sex has been calculated. 

In [18]:
median_age_scl = titanic_df[["Sex", "Pclass", "Age"]].groupby(['Pclass','Sex']).median()
median_age_scl

Age
Pclass Sex         
1      female  35.0
       male    40.0
2      female  28.0
       male    30.0
3      female  21.5
       male    25.0

Upon observation, these calculated ages seem reasonable, but it is possible to do better. We can then test and find out which other columns have missing values and could help us estimate a more accurate age for each missing value. From the result below we can see that apart from Age, Cabin and Embarked are the only other columns that have null values.

In [19]:
titanic_df.isnull().any()

Survived    False
Pclass      False
Name        False
Sex         False
Age          True
SibSp       False
Parch       False
Ticket      False
Fare        False
Cabin        True
Embarked     True
dtype: bool

Since passengers that have parents or children aboard based on the variable column (Parch) would undoubtedly have different ages, using this variable to further split each passenger into buckets is a logical idea. It should be noted that when Parch is equal 0, it means that passenger is a child with a nanny. This means such passengers are bound to have lower ages. It then makes sense to use Parch as another variable to split and calculate our median age values.

In [20]:
median_ages = titanic_df.groupby(["Sex", "Pclass","Parch","Age"])['Age'].median()
median_ages

Sex     Pclass  Parch  Age 
female  1       0      16.0    16.0
                       17.0    17.0
                       18.0    18.0
                       19.0    19.0
                       21.0    21.0
                               ... 
male    3       2      26.0    26.0
                       40.5    40.5
                3      16.0    16.0
                4      40.0    40.0
                5      39.0    39.0
Name: Age, Length: 374, dtype: float64

From the above table, we can see that we get differnt values for each value of Parch across each class and Sex. We can get more accurate measures of the age using these calculated values. We can now proceed to write a funtion to use this calculated median age and replace it for each null value in our dataset.

In [21]:
# We first find the missing values in the Age column and store them in a variable:
missing_ages = titanic_df[titanic_df['Age'].isnull()]

# The function provides the median age value if there are is a null value in the Age column based on the sex, class and Parch variables.

def remove_null_ages(row):
    """function to check if the age is null and replace wth the median from 
    the median ages dataframe defined above""" 
    
    if pd.isnull(row['Age']):
        return median_ages[row['Sex'],row['Pclass'],row['Parch']]
    else:
        return row['Age']

# We then use the apply method to use this function on our dataset.
titanic_df['Age'] = titanic_df.apply(remove_null_ages, axis=1)

After running this function, we can check and verify to see if it ran properly and if there are any more null values in the age column.

In [22]:
titanic_df.Age.isnull().sum()

0